# Datenbereinigung

# Inhalt

* [Daten aus Scopus](#data_scopus)
   * [Deduplikation](#data_scopus_deduplikation)
   * [Titelbereinigung](#data_scopus_bereinigung)
* [Daten aus OpenAlex](#data_openalex)
  * [Deduplikation](#data_openalex_deduplikation)
  * [Titelbereinigung](#data_openalex_bereinigung)

In [1]:
# import pybliometrics
import pandas as pd
import matplotlib.pyplot as plt

# from pybliometrics.scopus import ScopusSearch, AbstractRetrieval

# pybliometrics.scopus.init()  # read API keys
pd.options.mode.copy_on_write = True

# Daten für Publikation: für Scopus wird nur die id geseichert
# AUF False setzen, damit alle Daten aus Scopus gespeichert werden
publication = True 

In [2]:
import jupyter_black

jupyter_black.load()

In [3]:
%load_ext watermark
%watermark -v --iversions

Python implementation: CPython
Python version       : 3.11.2
IPython version      : 8.5.0

matplotlib   : 3.6.3
pandas       : 1.5.3
jupyter_black: 0.4.0



In [4]:
#  html text entfernen
# https://stackoverflow.com/questions/9662346/python-code-to-remove-html-tags-from-a-string

import re

# remove anything between < and >
CLEANR = re.compile("<.*?>")
# CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')


def cleanhtml(raw_html):
    """ Remove html text from string :string -> string"""
    cleantext = re.sub(CLEANR, "", raw_html)
    return cleantext

## Daten aus Scopus<a class="anchor" id="data_scopus"></a>

In [5]:
# Daten einlesen
data_scopus_2010 = pd.read_csv("../data/raw/data_scopus_2010.csv")
data_scopus_2020 = pd.read_csv("../data/raw/data_scopus_2020.csv")

print(data_scopus_2010.shape[0])
print(data_scopus_2020.shape[0])

4086
6505


In [6]:
data_scopus_2010.columns

Index(['eid', 'doi', 'pii', 'pubmed_id', 'title', 'subtype',
       'subtypeDescription', 'creator', 'afid', 'affilname',
       'affiliation_city', 'affiliation_country', 'author_count',
       'author_names', 'author_ids', 'author_afids', 'coverDate',
       'coverDisplayDate', 'publicationName', 'issn', 'source_id', 'eIssn',
       'aggregationType', 'volume', 'issueIdentifier', 'article_number',
       'pageRange', 'description', 'authkeywords', 'citedby_count',
       'openaccess', 'freetoread', 'freetoreadLabel', 'fund_acr', 'fund_no',
       'fund_sponsor'],
      dtype='object')

In [7]:
# für Scopus werden Publikationen vom Typ Conference Review nicht mit berücksichtigt
data_scopus_2010 = data_scopus_2010[
    data_scopus_2010["subtypeDescription"] != "Conference Review"
]
data_scopus_2020 = data_scopus_2020[
    data_scopus_2020["subtypeDescription"] != "Conference Review"
]

print(data_scopus_2010.shape[0])
print(data_scopus_2020.shape[0])

4030
6487


In [8]:
# für Scopus werden Publikationsquellen vom Typ Trade Journal nicht mit berücksichtigt
data_scopus_2010 = data_scopus_2010[
    data_scopus_2010["aggregationType"] != "Trade Journal"
]
data_scopus_2020 = data_scopus_2020[
    data_scopus_2020["aggregationType"] != "Trade Journal"
]

print(data_scopus_2010.shape[0])
print(data_scopus_2020.shape[0])

4021
6486


In [9]:
# Buchstaben einheitlich Kleinschreibung
data_scopus_2010["doi"] = data_scopus_2010["doi"].str.lower()
data_scopus_2020["doi"] = data_scopus_2020["doi"].str.lower()

data_scopus_2010["title"] = data_scopus_2010["title"].str.lower()
data_scopus_2020["title"] = data_scopus_2020["title"].str.lower()

data_scopus_2010["publicationName"] = data_scopus_2010["publicationName"].str.lower()
data_scopus_2020["publicationName"] = data_scopus_2020["publicationName"].str.lower()

### Deduplikation<a class="anchor" id="data_scopus_deduplikation"></a>

In [10]:
# Aufteilung in Daten mit und ohne DOI
data_scopus_2010_nodoi = data_scopus_2010[data_scopus_2010["doi"].isnull()]
data_scopus_2010_mitdoi = data_scopus_2010[data_scopus_2010["doi"].notnull()]

data_scopus_2020_nodoi = data_scopus_2020[data_scopus_2020["doi"].isnull()]
data_scopus_2020_mitdoi = data_scopus_2020[data_scopus_2020["doi"].notnull()]

print(
    f"Für 2010-2013 wurden {data_scopus_2010.shape[0]} Einträge in Scopus gefunden. Hiervon haben {data_scopus_2010_mitdoi.shape[0]} eine DOI und {data_scopus_2010_nodoi.shape[0]} keine DOI."
)
print(
    f"Für 2020-2023 wurden {data_scopus_2020.shape[0]} Einträge in Scopus gefunden. Hiervon haben {data_scopus_2020_mitdoi.shape[0]} eine DOI und {data_scopus_2020_nodoi.shape[0]} keine DOI."
)

Für 2010-2013 wurden 4021 Einträge in Scopus gefunden. Hiervon haben 3841 eine DOI und 180 keine DOI.
Für 2020-2023 wurden 6486 Einträge in Scopus gefunden. Hiervon haben 6392 eine DOI und 94 keine DOI.


In [11]:
# Prüfung anhand der DOI: gibt es Dopplungen?

duplicates_doi_data_scopus_2010 = data_scopus_2010_mitdoi[
    data_scopus_2010_mitdoi.duplicated("doi", keep=False)
]
duplicates_doi_data_scopus_2020 = data_scopus_2020_mitdoi[
    data_scopus_2020_mitdoi.duplicated("doi", keep=False)
]

print(
    f"Für 2010-2013 wurden {duplicates_doi_data_scopus_2010.shape[0]} Duplikate gefunden."
)
print(
    f"Für 2020-2023 wurden {duplicates_doi_data_scopus_2020.shape[0]} Duplikate gefunden."
)

# Duplikate abspeichern (zur manuellen Sichtung)
duplicates_doi_data_scopus_2010.to_csv(
    "../data/processed/duplicates_doi_data_scopus_2010.csv"
)
duplicates_doi_data_scopus_2020.to_csv(
    "../data/processed/duplicates_doi_data_scopus_2020.csv"
)

Für 2010-2013 wurden 8 Duplikate gefunden.
Für 2020-2023 wurden 8 Duplikate gefunden.


In [12]:
# Dopplungen entfernen
# zu entfernende Einträge, wurden manuell ermittelt
eid_2010_to_be_removed = [
    "2-s2.0-84892646502",
    "2-s2.0-84863704406",
    "2-s2.0-84897092929",
    "2-s2.0-84928953432",
]
eid_2020_to_be_removed = [
    "2-s2.0-85175387039",
    "2-s2.0-85175381055",
    "2-s2.0-85136175695",
    "2-s2.0-85112657532",
]

data_scopus_2010 = data_scopus_2010.query("eid not in @eid_2010_to_be_removed")
data_scopus_2020 = data_scopus_2020.query("eid not in @eid_2020_to_be_removed")

print(data_scopus_2010.shape[0])
print(data_scopus_2020.shape[0])

4017
6482


In [13]:
# Gibt es weitere Dopplungen? -> Prüfung aller Felder

duplicates_data_scopus_2010 = data_scopus_2010[data_scopus_2010.duplicated(keep=False)]
duplicates_data_scopus_2020 = data_scopus_2020[data_scopus_2020.duplicated(keep=False)]

print(
    f"Für 2010-2013 wurden {duplicates_data_scopus_2010.shape[0]} Duplikate gefunden."
)
print(
    f"Für 2020-2023 wurden {duplicates_data_scopus_2020.shape[0]} Duplikate gefunden."
)

Für 2010-2013 wurden 0 Duplikate gefunden.
Für 2020-2023 wurden 0 Duplikate gefunden.


In [14]:
# Gibt es weitere Dopplungen? -> Prüfung anhand der Felder 'title' und 'subTypeDescription'
# damit können bei gleichen Titel Journalartikel und Konferenzproceedings getrennt werden

duplicates_title_data_scopus_2010 = data_scopus_2010[
    data_scopus_2010.duplicated(subset=["title", "subtypeDescription"], keep=False)
]
duplicates_title_data_scopus_2020 = data_scopus_2020[
    data_scopus_2020.duplicated(subset=["title", "subtypeDescription"], keep=False)
]

print(
    f"Für 2010-2013 wurden {duplicates_title_data_scopus_2010.shape[0]} Duplikate gefunden."
)
print(
    f"Für 2020-2023 wurden {duplicates_title_data_scopus_2020.shape[0]} Duplikate gefunden."
)

# Duplikate abspeichern (zur manuellen Sichtung)
duplicates_title_data_scopus_2010.sort_values("title", inplace=True)
duplicates_title_data_scopus_2020.sort_values("title", inplace=True)
duplicates_title_data_scopus_2010.to_csv(
    "../data/processed/duplicates_title_data_scopus_2010.csv"
)
duplicates_title_data_scopus_2020.to_csv(
    "../data/processed/duplicates_title_data_scopus_2020.csv"
)

Für 2010-2013 wurden 33 Duplikate gefunden.
Für 2020-2023 wurden 14 Duplikate gefunden.


In [15]:
# Dopplungen entfernen
# zu entfernende Einträge, wurden manuell ermittelt
eid_2010_to_be_removed = [
    "2-s2.0-80755169735",
    "2-s2.0-84887704819",
    "2-s2.0-85120467391",
    "2-s2.0-84866000852",
    "2-s2.0-85030027095",
    "2-s2.0-84864102613",
    "2-s2.0-84893581577",
    "2-s2.0-79951619054",
    "2-s2.0-84898069435",
    "2-s2.0-84898070949",
    "2-s2.0-84898076009",
    "2-s2.0-79956106131",
]
eid_2020_to_be_removed = [
    "2-s2.0-85184127860",
    "2-s2.0-85120049152",
    "2-s2.0-85120036757",
    "2-s2.0-85091649541",
    "2-s2.0-85120050705",
    "2-s2.0-85145874050",
]

data_scopus_2010 = data_scopus_2010.query("eid not in @eid_2010_to_be_removed")
data_scopus_2020 = data_scopus_2020.query("eid not in @eid_2020_to_be_removed")

print(data_scopus_2010.shape[0])
print(data_scopus_2020.shape[0])

4006
6476


In [16]:
nodoi_2010 = data_scopus_2010[data_scopus_2010["doi"].isnull()]
nodoi_2020 = data_scopus_2020[data_scopus_2020["doi"].isnull()]

print(
    f"Finaler Datensatz für Scopus in 2010 - 2013: {data_scopus_2010.shape[0]} Einträge davon {nodoi_2010.shape[0]} ({nodoi_2010.shape[0]/data_scopus_2010.shape[0]*100:.2f} %) ohne DOI."
)
print(
    f"Finaler Datensatz für Scopus in 2020 - 2023: {data_scopus_2020.shape[0]} Einträge davon {nodoi_2020.shape[0]} ({nodoi_2020.shape[0]/data_scopus_2020.shape[0]*100:.2f} %) ohne DOI."
)

Finaler Datensatz für Scopus in 2010 - 2013: 4006 Einträge davon 169 (4.22 %) ohne DOI.
Finaler Datensatz für Scopus in 2020 - 2023: 6476 Einträge davon 88 (1.36 %) ohne DOI.


### Titelbereinigung<a class="anchor" id="data_scopus_bereinigung"></a>

Für die Einträge ohne DOI wird vorallem der Titel genutzt. Hier kann es vorallem durch Formelzeichen zu uneinheitlichen Schreibweisen kommen, was das Matching erschwert.
Hier findet sich u.a. html-Notation wie: hfo<inf>2</inf>. 

In [ ]:
title_to_fix_2010 = data_scopus_2010[data_scopus_2010["title"].str.contains("<inf>")]
title_to_fix_2020 = data_scopus_2020[data_scopus_2020["title"].str.contains("<inf>")]

pd.options.display.max_colwidth = 200
print(title_to_fix_2010["title"])
print(title_to_fix_2020["title"])

In [18]:
data_scopus_2010["title"] = data_scopus_2010["title"].apply(cleanhtml)
data_scopus_2020["title"] = data_scopus_2020["title"].apply(cleanhtml)

print(data_scopus_2010[data_scopus_2010["title"].str.contains("<inf>")])
print(data_scopus_2020[data_scopus_2020["title"].str.contains("<inf>")])

Empty DataFrame
Columns: [eid, doi, pii, pubmed_id, title, subtype, subtypeDescription, creator, afid, affilname, affiliation_city, affiliation_country, author_count, author_names, author_ids, author_afids, coverDate, coverDisplayDate, publicationName, issn, source_id, eIssn, aggregationType, volume, issueIdentifier, article_number, pageRange, description, authkeywords, citedby_count, openaccess, freetoread, freetoreadLabel, fund_acr, fund_no, fund_sponsor]
Index: []

[0 rows x 36 columns]
Empty DataFrame
Columns: [eid, doi, pii, pubmed_id, title, subtype, subtypeDescription, creator, afid, affilname, affiliation_city, affiliation_country, author_count, author_names, author_ids, author_afids, coverDate, coverDisplayDate, publicationName, issn, source_id, eIssn, aggregationType, volume, issueIdentifier, article_number, pageRange, description, authkeywords, citedby_count, openaccess, freetoread, freetoreadLabel, fund_acr, fund_no, fund_sponsor]
Index: []

[0 rows x 36 columns]


In [19]:
# Nachname ErstautorIn in separater Spalte
data_scopus_2010["name_1st_author"] = (
    data_scopus_2010["author_names"].str.split(",").str[0]
)
data_scopus_2020["name_1st_author"] = (
    data_scopus_2020["author_names"].str.split(",").str[0]
)

In [20]:
# Publikationsjahr in separater Spalte
data_scopus_2010["publication_year"] = (
    data_scopus_2010["coverDate"].str.split("-").str[0]
)
data_scopus_2020["publication_year"] = (
    data_scopus_2020["coverDate"].str.split("-").str[0]
)

## Daten aus OpenAlex<a class="anchor" id="data_openalex"></a>

In [21]:
# Daten einlesen
data_openalex_2010 = pd.read_csv("../data/raw/data_openalex_2010.csv")
data_openalex_2020 = pd.read_csv("../data/raw/data_openalex_2020.csv")

print(data_openalex_2010.shape[0])
print(data_openalex_2020.shape[0])

5530
7622


In [22]:
# für OpenAlex werden Publikationen vom Typ paratext, preprint, dissertation, grant, pee-review, standard nicht mit berücksichtigt
types_not_considered = [
    "paratext",
    "preprint",
    "grant",
    "dissertation",
    "dataset",
    "peer-review",
    "standard",
]

data_openalex_2010 = data_openalex_2010.query("type not in @types_not_considered")
data_openalex_2020 = data_openalex_2020.query("type not in @types_not_considered")

print(data_openalex_2010.shape[0])
print(data_openalex_2020.shape[0])

5266
6968


In [23]:
# Buchstaben einheitlich Kleinschreibung
data_openalex_2010["doi"] = data_openalex_2010["doi"].str.lower()
data_openalex_2020["doi"] = data_openalex_2020["doi"].str.lower()

data_openalex_2010["title"] = data_openalex_2010["title"].str.lower()
data_openalex_2020["title"] = data_openalex_2020["title"].str.lower()

data_openalex_2010["source_title"] = data_openalex_2010["source_title"].str.lower()
data_openalex_2020["source_title"] = data_openalex_2020["source_title"].str.lower()

In [24]:
# DOI bereinigen
# DOI in OpenAlex starten mit  https://doi.org/, in Scopus ist dies bereits entfernt

data_openalex_2010["doi"] = data_openalex_2010["doi"].str.lstrip("https://doi.org/")
data_openalex_2020["doi"] = data_openalex_2020["doi"].str.lstrip("https://doi.org/")

### Deduplikation<a class="anchor" id="data_openalex_deduplikation"></a>

In [25]:
# Prüfung anhand der ID: Gibt es Dopplungen?
# Dopplungen sind identische Einträge, daher keine manuelle Entfernung
# Herkunft unklar

# Duplikate finden
duplicates_id_data_openalex_2010 = data_openalex_2010[
    data_openalex_2010.duplicated("id", keep=False)
]
duplicates_id_data_openalex_2020 = data_openalex_2020[
    data_openalex_2020.duplicated("id", keep=False)
]

print(
    f"Für 2010-2013 wurden {duplicates_id_data_openalex_2010.shape[0]} Duplikate gefunden."
)
print(
    f"Für 2020-2023 wurden {duplicates_id_data_openalex_2020.shape[0]} Duplikate gefunden."
)

# Duplikate abspeichern (zur manuellen Sichtung)
# duplicates_id_data_openalex_2010.to_csv(
#     "data/processed/duplicates_id_data_openalex_2010.csv", encoding="utf-8"
# )
# duplicates_id_data_openalex_2020.to_csv(
#     "data/processed/duplicates_id_data_openalex_2020.csv", encoding="utf-8"
# )

# Duplikate entfernen
data_openalex_2010.drop_duplicates(subset=["id"], inplace=True)
data_openalex_2020.drop_duplicates(subset=["id"], inplace=True)

print(data_openalex_2010.shape[0])
print(data_openalex_2020.shape[0])

Für 2010-2013 wurden 26 Duplikate gefunden.
Für 2020-2023 wurden 24 Duplikate gefunden.
5253
6956


In [26]:
# Aufteilung in Daten mit und ohne DOI
data_openalex_2010_nodoi = data_openalex_2010[data_openalex_2010["doi"].isnull()]
data_openalex_2010_mitdoi = data_openalex_2010[data_openalex_2010["doi"].notnull()]

data_openalex_2020_nodoi = data_openalex_2020[data_openalex_2020["doi"].isnull()]
data_openalex_2020_mitdoi = data_openalex_2020[data_openalex_2020["doi"].notnull()]

print(
    f"Für 2010-2013 wurden {data_openalex_2010.shape[0]} Einträge in OpenAlex gefunden. Hiervon haben {data_openalex_2010_mitdoi.shape[0]} eine DOI und {data_openalex_2010_nodoi.shape[0]} keine DOI."
)
print(
    f"Für 2020-2023 wurden {data_openalex_2020.shape[0]} Einträge in OpenAlex gefunden. Hiervon haben {data_openalex_2020_mitdoi.shape[0]} eine DOI und {data_openalex_2020_nodoi.shape[0]} keine DOI."
)

Für 2010-2013 wurden 5253 Einträge in OpenAlex gefunden. Hiervon haben 4603 eine DOI und 650 keine DOI.
Für 2020-2023 wurden 6956 Einträge in OpenAlex gefunden. Hiervon haben 6780 eine DOI und 176 keine DOI.


In [27]:
# überprüfung: alle DOIs starten mit 10.1
print(
    f"2010-2013: {data_openalex_2010_mitdoi.shape[0]}, {data_openalex_2010_mitdoi[data_openalex_2010_mitdoi['doi'].str.startswith('10.')].shape[0]}"
)
print(
    f"2020-2023: {data_openalex_2020_mitdoi.shape[0]}, {data_openalex_2020_mitdoi[data_openalex_2020_mitdoi['doi'].str.startswith('10.')].shape[0]}"
)

2010-2013: 4603, 4603
2020-2023: 6780, 6780


In [28]:
# Prüfung anhand der DOI: Gibt es Dopplungen?

duplicates_doi_data_openalex_2010 = data_openalex_2010_mitdoi[
    data_openalex_2010_mitdoi.duplicated("doi", keep=False)
]
duplicates_doi_data_openalex_2020 = data_openalex_2020_mitdoi[
    data_openalex_2020_mitdoi.duplicated("doi", keep=False)
]

print(
    f"Für 2010-2013 wurden {duplicates_doi_data_openalex_2010.shape[0]} Duplikate gefunden."
)
print(
    f"Für 2020-2023 wurden {duplicates_doi_data_openalex_2020.shape[0]} Duplikate gefunden."
)

# Duplikate abspeichern
# duplicates_doi_data_openalex_2010.to_csv("data/processed/duplicates_doi_data_openalex_2010.csv", mode='x')
# duplicates_doi_data_openalex_2020.to_csv("data/processed/duplicates_doi_data_openalex_2020.csv", mode='x')

Für 2010-2013 wurden 0 Duplikate gefunden.
Für 2020-2023 wurden 0 Duplikate gefunden.


In [29]:
# Prüfung anhand des Titels: Gibt es Dopplungen?
duplicates_title_data_openalex_2010 = data_openalex_2010[
    data_openalex_2010.duplicated(subset=["title", "type"], keep=False)
]
duplicates_title_data_openalex_2020 = data_openalex_2020[
    data_openalex_2020.duplicated(subset=["title", "type"], keep=False)
]

print(
    f"Für 2010-2013 wurden {duplicates_title_data_openalex_2010.shape[0]} Duplikate gefunden."
)
print(
    f"Für 2020-2023 wurden {duplicates_title_data_openalex_2020.shape[0]} Duplikate gefunden."
)

# Duplikate abspeichern (zur manuellen Sichtung)
# duplicates_title_data_openalex_2010.sort_values("title", inplace=True)
# duplicates_title_data_openalex_2020.sort_values("title", inplace=True)
# duplicates_title_data_openalex_2010.to_csv(
#     "data/processed/duplicates_title_data_openalex_2010.csv"
# )
# duplicates_title_data_openalex_2020.to_csv(
#     "data/processed/duplicates_title_data_openalex_2020.csv"
# )

Für 2010-2013 wurden 214 Duplikate gefunden.
Für 2020-2023 wurden 248 Duplikate gefunden.


In [30]:
# Datensätze ohne DOI
duplicates_title_data_openalex_2010[
    duplicates_title_data_openalex_2010["doi"].notnull()
].shape[0]

188

In [31]:
# Datensätze ohne DOI
duplicates_title_data_openalex_2020[
    duplicates_title_data_openalex_2020["doi"].notnull()
].shape[0]

236

In [32]:
# Dopplungen für das Journal Japanese Journal of Applied Physics entfernen (doppelte Einträge mit unterschiedlichen DOI Prefixen)

data_openalex_2010 = data_openalex_2010[
    ~data_openalex_2010["doi"].str.startswith("10.7567", na=False)
]
data_openalex_2020 = data_openalex_2020[
    ~data_openalex_2020["doi"].str.startswith("10.7567", na=False)
]

print(data_openalex_2010.shape[0])
print(data_openalex_2020.shape[0])

5172
6951


In [33]:
# Article des Preprintservers SSRN werden nicht mit berücksichtigt
# DOI starten mit 10.2139/ssrn

data_openalex_2010 = data_openalex_2010[
    ~data_openalex_2010["doi"].str.startswith("10.2139/ssrn", na=False)
]
data_openalex_2020 = data_openalex_2020[
    ~data_openalex_2020["doi"].str.startswith("10.2139/ssrn", na=False)
]

print(data_openalex_2010.shape[0])
print(data_openalex_2020.shape[0])

5172
6806


In [34]:
# Meeting Abstract der ECS transactions entfernen (Proceedings bleiben erhalten)
# DOI starten mit 10.1149/ma

data_openalex_2010 = data_openalex_2010[
    ~data_openalex_2010["doi"].str.startswith("10.1149/ma", na=False)
]
data_openalex_2020 = data_openalex_2020[
    ~data_openalex_2020["doi"].str.startswith("10.1149/ma", na=False)
]

print(data_openalex_2010.shape[0])
print(data_openalex_2020.shape[0])

5028
6294


In [35]:
# Beträge zur Asia Communications and Photonics Conference sind teilweise doppelt enthalten
# Beiträge starten mit DOI 10.1364/acp und 10.1364/acpc., führen aber zum gleichen Dokument
# DOI mit Beginn 10.1364/acp. werden entfernt, wenn Dopplung vorhanden (manuell)

doi_2010_to_be_removed = [
    "10.1364/acp.2013.aw4c.5",
    "10.1364/acp.2013.af2d.29",
    "10.1364/acp.2013.af2b.44",
]
# keine Dopplung hier für 2020

data_openalex_2010 = data_openalex_2010.query("doi not in @doi_2010_to_be_removed")

print(data_openalex_2010.shape[0])

5025


In [36]:
# Welche der beiden DOI-Varianten sind in Scopus zu finden?
data_scopus_2010[
    data_scopus_2010["doi"] == "10.1364/acp"
]  # weder noch in Scopus vorhanden

,eid,doi,pii,pubmed_id,title,subtype,subtypeDescription,creator,afid,affilname,...,authkeywords,citedby_count,openaccess,freetoread,freetoreadLabel,fund_acr,fund_no,fund_sponsor,name_1st_author,publication_year


In [37]:
# Dopplungen, wo nur ein Eintrag eine DOI besitzt -> dieser wurde behalten und der Eintrag ohne DOI entfernt (manuell)

# Einträge ohne DOI, welche behalten werden -> alle sonstigen ohne DOI werden entfernt
id_2010_to_keep_nodoi = [
    "https://openalex.org/W1838703554",
    "https://openalex.org/W1531779540",
]
# für 2020 werden alle Einträge ohne DOI entfernt, da hier immer ein weiterer Eintrag mit DOI vorhanden ist

duplicates_nodoi_2010 = duplicates_title_data_openalex_2010[
    duplicates_title_data_openalex_2010["doi"].isnull()
]
duplicates_nodoi_2020 = duplicates_title_data_openalex_2020[
    duplicates_title_data_openalex_2020["doi"].isnull()
]

id_2010_to_be_removed = [
    x for x in duplicates_nodoi_2010["id"] if x not in id_2010_to_keep_nodoi
]
id_2020_to_be_removed = duplicates_nodoi_2020["id"].to_list()

print(id_2010_to_be_removed)
print(id_2020_to_be_removed)

data_openalex_2010 = data_openalex_2010.query("id not in @id_2010_to_be_removed")
data_openalex_2020 = data_openalex_2020.query("id not in @id_2020_to_be_removed")

# Finale Datensatzgröße
print(data_openalex_2010.shape[0])
print(data_openalex_2020.shape[0])

['https://openalex.org/W1578420299', 'https://openalex.org/W3142023407', 'https://openalex.org/W1483626592', 'https://openalex.org/W3042542394', 'https://openalex.org/W3150415440', 'https://openalex.org/W2295937238', 'https://openalex.org/W601724471', 'https://openalex.org/W2244171064', 'https://openalex.org/W2139056038', 'https://openalex.org/W3147880047', 'https://openalex.org/W2784558900', 'https://openalex.org/W3144557893', 'https://openalex.org/W179212451', 'https://openalex.org/W1605062476', 'https://openalex.org/W3151139258', 'https://openalex.org/W3143281919', 'https://openalex.org/W2347179810', 'https://openalex.org/W2969997526', 'https://openalex.org/W2093006866', 'https://openalex.org/W2134029727', 'https://openalex.org/W3006286394', 'https://openalex.org/W3181416261', 'https://openalex.org/W2752407893', 'https://openalex.org/W2994728064']
['https://openalex.org/W3030573482', 'https://openalex.org/W3209633430', 'https://openalex.org/W4315751225', 'https://openalex.org/W31491

In [38]:
# Prüfung anhand des Titels: Welche Dopplungen sind noch verblieben?
duplicates_title_data_openalex_2010 = data_openalex_2010[
    data_openalex_2010.duplicated(
        subset=["title", "type", "publication_year"], keep=False
    )
]
duplicates_title_data_openalex_2020 = data_openalex_2020[
    data_openalex_2020.duplicated(
        subset=["title", "type", "publication_year"], keep=False
    )
]

print(
    f"Für 2010-2013 wurden {duplicates_title_data_openalex_2010.shape[0]} Duplikate gefunden."
)
print(
    f"Für 2020-2023 wurden {duplicates_title_data_openalex_2020.shape[0]} Duplikate gefunden."
)

# Duplikate abspeichern (zur manuellen Sichtung)
# duplicates_title_data_openalex_2010.sort_values("title", inplace=True)
# duplicates_title_data_openalex_2020.sort_values("title", inplace=True)
# duplicates_title_data_openalex_2010.to_csv(
#     "../data/processed/duplicates_title2_data_openalex_2010.csv"
# )
# duplicates_title_data_openalex_2020.to_csv(
#     "../data/processed/duplicates_title2_data_openalex_2020.csv"
# )

Für 2010-2013 wurden 44 Duplikate gefunden.
Für 2020-2023 wurden 81 Duplikate gefunden.


Die verbleibenden Dopplungen werden beibehalten. 
Alle Publikationen haben eine DOI.

In [39]:
nodoi_2010 = data_openalex_2010[data_openalex_2010["doi"].isnull()]
nodoi_2020 = data_openalex_2020[data_openalex_2020["doi"].isnull()]

print(
    f"Finaler Datensatz für OpenAlex in 2010 - 2013: {data_openalex_2010.shape[0]} Einträge davon {nodoi_2010.shape[0]} ({nodoi_2010.shape[0]/data_openalex_2010.shape[0]*100:.2f} %) ohne DOI."
)
print(
    f"Finaler Datensatz für OpenAlex in 2020 - 2023: {data_openalex_2020.shape[0]} Einträge davon {nodoi_2020.shape[0]} ({nodoi_2020.shape[0]/data_openalex_2020.shape[0]*100:.2f} %) ohne DOI."
)

Finaler Datensatz für OpenAlex in 2010 - 2013: 5001 Einträge davon 626 (12.52 %) ohne DOI.
Finaler Datensatz für OpenAlex in 2020 - 2023: 6282 Einträge davon 164 (2.61 %) ohne DOI.


### Titelbereinigung<a class="anchor" id="data_openalex_bereinigung"></a>

Für die Einträge ohne DOI wird vorallem der Titel genutzt. Hier kann es durch Formelzeichen zu uneinheitlichen Schreibweisen kommen, was das Matching erschwert.
Hier findet sich u.a. html-Notation wie: hfo<inf>2</inf>. 

In [40]:
title_to_fix_2010 = data_openalex_2010[
    data_openalex_2010["title"].str.contains("<inf>")
]
title_to_fix_2020 = data_openalex_2020[
    data_openalex_2020["title"].str.contains("<inf>")
]

pd.options.display.max_colwidth = 200
print(title_to_fix_2010["title"])
print(title_to_fix_2020["title"])

5144    high-performance enhancement-mode in<inf>0.53</inf>ga<inf>0.47</inf>as surface channels n-mosfet with thin in<inf>0.2</inf>ga<inf>0.8</inf>as capping and laser anneal effect
5169                                                                                                                  band alignment of high-k dielectric on sio<inf>2</inf>/si stack
Name: title, dtype: object
Series([], Name: title, dtype: object)


In [41]:
title_to_fix_2010 = data_openalex_2010[data_openalex_2010["title"].str.contains("xml")]
title_to_fix_2020 = data_openalex_2020[data_openalex_2020["title"].str.contains("xml")]

In [42]:
data_openalex_2010["title"] = data_openalex_2010["title"].apply(cleanhtml)
data_openalex_2020["title"] = data_openalex_2020["title"].apply(cleanhtml)

In [43]:
print(data_openalex_2010[data_openalex_2010["title"].str.contains("xml")])
print(data_openalex_2020[data_openalex_2020["title"].str.contains("xml")])

                                    id                         doi  \
3858  https://openalex.org/W2325674568  10.1103/physrevb.87.035418   

                                                                                                                         title  \
3858  structural phase transition and spontaneous interface reconstruction in la2/3ca1/3mno<mml:math xmlns:mml="http://www.w3…   

         type publication_date  publication_year volume issue first_page  \
3858  article       2013-01-22              2013     87     3        NaN   

     last_page  \
3858       NaN   

                                                                                        authors  \
3858  Stuart Turner, Oleg I. Lebedev, Johan Verbeeck, Kai Gehrke, V. Moshnyaga, G. Van Tendeloo   

     keywords  \
3858      NaN   

                                                                                                                                                                              

In [44]:
data_openalex_2010[data_openalex_2010["title"].str.contains("&lt;")].iloc[0]

id                                                                                                                                                                                         https://openalex.org/W2320409272
doi                                                                                                                                                                                                   10.1166/jnn.2011.5102
title                                                                                                                        growth of tio&lt;sub&gt;2&lt;/sub&gt; with thermal and plasma enhanced atomic layer deposition
type                                                                                                                                                                                                                article
publication_date                                                                                                        

In [45]:
# remove other html stuff
# 2010
data_openalex_2010["title"] = data_openalex_2010["title"].str.replace(
    "&lt;sub&gt;", " "
)
data_openalex_2010["title"] = data_openalex_2010["title"].str.replace(
    "&lt;/sub&gt;", " "
)
data_openalex_2010["title"] = data_openalex_2010["title"].str.replace(
    "&lt;inf&gt;", " "
)
data_openalex_2010["title"] = data_openalex_2010["title"].str.replace(
    "&lt;/inf&gt;", " "
)

# 2020
data_openalex_2020["title"] = data_openalex_2020["title"].str.replace(
    "&lt;sub&gt;", " "
)
data_openalex_2020["title"] = data_openalex_2020["title"].str.replace(
    "&lt;/sub&gt;", " "
)
data_openalex_2020["title"] = data_openalex_2020["title"].str.replace(
    "&lt;inf&gt;", " "
)
data_openalex_2020["title"] = data_openalex_2020["title"].str.replace(
    "&lt;/inf&gt;", " "
)

In [46]:
data_openalex_2010["name_1st_author"] = (
    data_openalex_2010["authors"].str.split(",").str[0].str.split(" ").str[-1]
)
data_openalex_2020["name_1st_author"] = (
    data_openalex_2020["authors"].str.split(",").str[0].str.split(" ").str[-1]
)

In [47]:
# Speichern der bereinigten Datensätze

## Scopus
if publication:
    # Daten für Veröffentlichung nur mit eid
    data_scopus_2010["eid"].to_csv(
        "data/processed/data_scopus_2010_cleaned_pub.csv", index=False
    )
    data_scopus_2020["eid"].to_csv(
        "data/processed/data_scopus_2020_cleaned_pub.csv", index=False
    )
else:
    data_scopus_2010.to_csv(
        "data/processed/data_scopus_2010_cleaned.csv", index=False
    )
    data_scopus_2020.to_csv(
        "data/processed/data_scopus_2020_cleaned.csv", index=False
    )

## OpenAlex
data_openalex_2010.to_csv(
    "data/processed/data_openalex_2010_cleaned.csv", index=False
)
data_openalex_2020.to_csv(
    "data/processed/data_openalex_2020_cleaned.csv", index=False
)